## Preface  

Most classification algorithms will only perform optimally when the number of samples of each class is roughly the same. Highly skewed datasets, where the minority is heavily outnumbered by one or more classes, have proven to be a challenge while at the same time becoming more and more common.  
https://pypi.python.org/pypi/imbalanced-learn 

Imbalanced data is commonly observed in the real-world setting, real-fraudulent transaction, healthy-infected patient, to name a few. Incautious attempts of machine learning techniques on the problem could give very bad results/prediction. Furthermore, the inappropriate metric of performance measurement provides wrong conclusion due to the nature of the evaluation method. In this project, we try to tackle such problem and compare the model improvement.  


## Preprocessing

### Sampling techniques
1. Undersampling
    - Random undersampling
    - Cluster Centroids
    - Near Miss
2. Oversampling
    - Random oversampling: generate new samples by random resampling with replacement of under represented class
    - Synthetic Minority Oversampling (SMOTE)
3. Combined over and under sampling
    - SMOTEENN
    - SMOTETomek  

### Training techniques  
1. Class weighting
2. Sample weighting

## Import the Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display
%matplotlib inline

from imblearn.under_sampling import ClusterCentroids, NearMiss, RandomUnderSampler
from imblearn.over_sampling import SMOTE, RandomOverSampler

from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.ensemble import BalanceCascade

from sklearn.metrics import recall_score, accuracy_score, confusion_matrix, \
f1_score, precision_score, auc, roc_auc_score, roc_curve, precision_recall_curve
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler


## Load the dataset

In [2]:
df = pd.read_csv('./input/creditcard.csv')
display(df.head(5))

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
display(df.describe())

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.758743e-12,-8.252298e-13,-9.636929e-13,8.316157e-13,1.591952e-13,4.247354e-13,-3.050180e-13,8.693344e-14,-1.179712e-12,...,-3.406543e-13,-5.713163e-13,-9.725303e-13,1.464139e-12,-6.989087e-13,-5.615260e-13,3.332112e-12,-3.518886e-12,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [4]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [5]:
df['Amount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1, 1))
df = df.drop(['Time'], axis = 1)
display(df.head(5))

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403,0


## Split the data 

In [6]:
X = df.iloc[:, df.columns != 'Class']
y = df.iloc[:, df.columns == 'Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)

print(X_train.shape)
print(X_test.shape)

(199364, 29)
(85443, 29)


In [7]:
def transform(transformer, X, y):
    print('Transforming {}'.format(transformer.__class__.__name__))
    X_resampled, y_resampled = transformer.fit_sample(X.values, y.values.ravel())
    return transformer.__class__.__name__, pd.DataFrame(X_resampled), pd.DataFrame(y_resampled)

def benchmark(sampling_type, X, y):
    lr = LogisticRegression(penalty = 'l1')
    param_grid = {'C': [0.1,1,10]}
    g_search = GridSearchCV(estimator = lr, param_grid = param_grid, scoring = 'accuracy',
                            cv = 4, verbose = 2)
    g_search = g_search.fit(X.values, y.values.ravel())
    return sampling_type, g_search.best_score_, g_search.best_params_['C']


In [8]:
datasets = []
datasets.append(('base', X_train, y_train))
datasets.append(transform(RandomUnderSampler(), X_train, y_train))
datasets.append(transform(NearMiss(n_jobs=-1), X_train, y_train))
datasets.append(transform(RandomOverSampler(), X_train, y_train))
datasets.append(transform(SMOTE(n_jobs=-1), X_train, y_train))

## It is computational demanding for larger data sets.
## datasets.append(transform(SMOTEENN(), X_train, y_train))
## datasets.append(transform(SMOTETomek(), X_train, y_train))

Transforming RandomUnderSampler
Transforming NearMiss
Transforming RandomOverSampler
Transforming SMOTE


In [9]:
display([item[0] for item in datasets])

['base', 'RandomUnderSampler', 'NearMiss', 'RandomOverSampler', 'SMOTE']

### Determine Hyper-Parameters

In [10]:
## sampling_type, g_search.best_score_, g_search.best_params_['C']
benchmark_scores = []
for sample_type, X, y in datasets:
    print('------')
    print('{}'.format(sample_type))
    benchmark_scores.append(benchmark(sample_type,X,y))
    print('------')


------
base
Fitting 4 folds for each of 3 candidates, totalling 12 fits
[CV] C=0.1 ...........................................................
[CV] ............................................ C=0.1, total=   3.6s
[CV] C=0.1 ...........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.7s remaining:    0.0s


[CV] ............................................ C=0.1, total=   3.9s
[CV] C=0.1 ...........................................................
[CV] ............................................ C=0.1, total=   3.8s
[CV] C=0.1 ...........................................................
[CV] ............................................ C=0.1, total=   3.8s
[CV] C=1 .............................................................
[CV] .............................................. C=1, total=  14.1s
[CV] C=1 .............................................................
[CV] .............................................. C=1, total=  15.9s
[CV] C=1 .............................................................
[CV] .............................................. C=1, total=  12.4s
[CV] C=1 .............................................................
[CV] .............................................. C=1, total=  11.7s
[CV] C=10 ............................................................
[CV] .

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  2.5min finished


------
------
RandomUnderSampler
Fitting 4 folds for each of 3 candidates, totalling 12 fits
[CV] C=0.1 ...........................................................
[CV] ............................................ C=0.1, total=   0.0s
[CV] C=0.1 ...........................................................
[CV] ............................................ C=0.1, total=   0.0s
[CV] C=0.1 ...........................................................
[CV] ............................................ C=0.1, total=   0.0s
[CV] C=0.1 ...........................................................
[CV] ............................................ C=0.1, total=   0.0s
[CV] C=1 .............................................................
[CV] .............................................. C=1, total=   0.0s
[CV] C=1 .............................................................
[CV] .............................................. C=1, total=   0.0s
[CV] C=1 ..............................................

...

------
------
SMOTE
Fitting 4 folds for each of 3 candidates, totalling 12 fits
[CV] C=0.1 ...........................................................
[CV] ............................................ C=0.1, total=  29.6s
[CV] C=0.1 ...........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   29.7s remaining:    0.0s


[CV] ............................................ C=0.1, total=  26.0s
[CV] C=0.1 ...........................................................
[CV] ............................................ C=0.1, total=  23.7s
[CV] C=0.1 ...........................................................
[CV] ............................................ C=0.1, total=  24.0s
[CV] C=1 .............................................................
[CV] .............................................. C=1, total=  25.4s
[CV] C=1 .............................................................
[CV] .............................................. C=1, total=  41.8s
[CV] C=1 .............................................................
[CV] .............................................. C=1, total=  24.1s
[CV] C=1 .............................................................
[CV] .............................................. C=1, total=  24.9s
[CV] C=10 ............................................................
[CV] .

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  5.4min finished


------


In [11]:
display(benchmark_scores)

[('base', 0.9992175116871652, 1),
 ('RandomUnderSampler', 0.9367469879518072, 1),
 ('NearMiss', 0.9653614457831325, 1),
 ('RandomOverSampler', 0.9488499336790064, 0.1),
 ('SMOTE', 0.9462799951766551, 1)]

### Evaluation models

In [12]:
print (X_train.shape)

(199364, 29)


In [14]:
## http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.predict_proba

## http://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html#sphx-glr-auto-examples-model-selection-plot-precision-recall-py
from sklearn.metrics import average_precision_score

scores = []
for sampling_type, score, param in benchmark_scores:
    print('Training on {}'.format(sampling_type))
    lr = LogisticRegression(penalty='l1', C = param)
    for s_type, X, y in datasets:
        if s_type == sampling_type:
            lr.fit(X.values, y.values.ravel())
            y_pred_class = lr.predict(X_test.values)
            
            ## Probability estimates
            y_pred_prob = lr.predict_proba(X_test.values)
            
            ## Predict confidence scores for samples.
            y_pred_confi = lr.decision_function(X_test.values)
            
            fpr, tpr, threshold = roc_curve(y_test.values.ravel(), y_pred_class)
            prec, recall, thres = precision_recall_curve(y_test.values.ravel(), y_pred_confi)

            
            ## average_precision = average_precision_score(y_test.values.ravel(), y_pred_confi)

            
            scores.append((sampling_type,
                           accuracy_score(y_test.values.ravel(), y_pred_class),
                           f1_score(y_test.values.ravel(), y_pred_class),
                           precision_score(y_test.values.ravel(), y_pred_class),
                           recall_score(y_test.values.ravel(), y_pred_class),
                           average_precision_score(y_test.values.ravel(), y_pred_confi),
                           auc(fpr, tpr),
                           auc(prec, recall, reorder = True),
                           confusion_matrix(y_test.values.ravel(), y_pred_class)))  ## tn, fp, fn, tp 

Training on base
Training on RandomUnderSampler
Training on NearMiss
Training on RandomOverSampler
Training on SMOTE


In [15]:
sampling_results = pd.DataFrame(scores, columns=['Sampling Type', 'accuracy', 'f1', 'precision',
                                                 'recall', 'average_precision', 
                                                 'auc_roc', 'auc_pr', 'confusion_matrix'])
display(sampling_results)

,Sampling Type,accuracy,f1,precision,recall,average_precision,auc_roc,auc_pr,confusion_matrix
0,base,0.999017,0.693431,0.833333,0.59375,0.750553,0.796764,0.743638,"[[85264, 19], [65, 95]]"
1,RandomUnderSampler,0.969512,0.100794,0.053343,0.91250,0.473387,0.941059,0.510237,"[[82692, 2591], [14, 146]]"
2,NearMiss,0.425582,0.006277,0.003148,0.96875,0.043496,0.696656,0.079994,"[[36208, 49075], [5, 155]]"
3,RandomOverSampler,0.975247,0.121313,0.064976,0.91250,0.768724,0.943932,0.758841,"[[83182, 2101], [14, 146]]"
4,SMOTE,0.973573,0.115204,0.061455,0.91875,0.767649,0.946213,0.757026,"[[83038, 2245], [13, 147]]"


### Weighted Class

In [16]:
lr = LogisticRegression(penalty = 'l1', class_weight = 'balanced')
lr.fit(X_train.values, y_train.values.ravel())

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [17]:
scores = []
y_pred_class = lr.predict(X_test.values)
y_pred_proba = lr.predict_proba(X_test.values)
y_pred_confi = lr.decision_function(X_test.values)
fpr, tpr, thresholds = roc_curve(y_test.values.ravel(), y_pred_class)
precision, recall, thres = precision_recall_curve(y_test.values.ravel(), y_pred_confi)
scores.append(("weighted_base", 
               accuracy_score(y_test.values.ravel(),y_pred_class), 
               f1_score(y_test.values.ravel(),y_pred_class),
               precision_score(y_test.values.ravel(),y_pred_class),
               recall_score(y_test.values.ravel(),y_pred_class),
               average_precision_score(y_test.values.ravel(), y_pred_confi),
               auc(fpr, tpr),
               auc(precision, recall, reorder=True),
               confusion_matrix(y_test.values.ravel(),y_pred_class)))
scores = pd.DataFrame(scores, columns = ['Sampling Type', 'accuracy', 'f1', 'precision',
                                                 'recall', 'average_precision', 
                                                 'auc_roc', 'auc_pr', 'confusion_matrix'])

In [18]:
results = sampling_results.append(scores)
display(results)


,Sampling Type,accuracy,f1,precision,recall,average_precision,auc_roc,auc_pr,confusion_matrix
0,base,0.999017,0.693431,0.833333,0.59375,0.750553,0.796764,0.743638,"[[85264, 19], [65, 95]]"
1,RandomUnderSampler,0.969512,0.100794,0.053343,0.91250,0.473387,0.941059,0.510237,"[[82692, 2591], [14, 146]]"
2,NearMiss,0.425582,0.006277,0.003148,0.96875,0.043496,0.696656,0.079994,"[[36208, 49075], [5, 155]]"
3,RandomOverSampler,0.975247,0.121313,0.064976,0.91250,0.768724,0.943932,0.758841,"[[83182, 2101], [14, 146]]"
4,SMOTE,0.973573,0.115204,0.061455,0.91875,0.767649,0.946213,0.757026,"[[83038, 2245], [13, 147]]"
0,weighted_base,0.975212,0.121162,0.064889,0.91250,0.769961,0.943915,0.759453,"[[83179, 2104], [14, 146]]"


## Reflection
- Undersampling leads to high recall, as long as a huge downside of precision
- SMOTE sampling and RandomOverSampler perform the best considering auc_roc and auc_pr with acceptable levels of false positives
- Class weighting could give comparable results to sampling techniques